In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:

def merge_datasets(path):	#takes the path and returns a merged dataset
    data_m = pd.read_csv(path+'movies.csv')
    data_r = pd.read_csv(path+'ratings.csv')
    data_u = pd.read_csv(path+'users.csv')

    movie_ratings = pd.merge(data_m, data_r)
    lens = pd.merge(movie_ratings, data_u)
    #print(lens.head())

    #most_rated = lens.groupby('title').size().sort_values(ascending=False)[:25]
    #print(most_rated)

    return lens

def create_user_item_pair(data):
    ## picks top k rated movies & top k users with most ratings
    ## selects subset containing only above users and movies
    ## creates (user, item) pair matrix with rating as cell value
    lens = data.copy()

    most_rated_users = lens.groupby('userId').size().sort_values(ascending=False)
    mru_keys = list(most_rated_users.keys())[:600]

    most_rated_movies = lens.groupby('itemId').size().sort_values(ascending=False)
    mrm_keys = list(most_rated_movies.keys())[:400]


    data = data.loc[data['userId'].isin(mru_keys) & data['itemId'].isin(mrm_keys)]
    data = data.pivot(index='userId', columns='itemId', values='rating')
    data = np.where(np.isnan(data),0, data)
    #print(data.head())
    return data

def create_matrix_I(data):
    I = data.copy()
    I = np.asarray(I)
    I = np.where(np.isnan(I), 0, 1)
    
    return I    


In [25]:
def create_user_user_similarity_matrix(data):
    uu_matrix = cosine_similarity(data)
    return uu_matrix
    
def create_prediction_matrix(A, uu_matrix):
    ## cf algorithm's predictions
    pred_matrix = np.zeros(A.shape)
    K = A.shape[0] #no. of users
    w = A.shape[1] #no. of items
    
    mean_ratings_of_users = []
    sum_of_user_similarity = []
    for i in range(K):
        mean = np.sum(A[i])/np.count_nonzero(A[i])
        mean_ratings_of_users.append(mean)
        sum_of_user_similarity.append(np.sum(uu_matrix[i]))
    for i in range(0, w): #items
        for a in range(0, K): #users
            sum = 0
            for u in range(K):
                sum += (A[u][i] - mean_ratings_of_users[u]) * uu_matrix[a][u]
            sum /= sum_of_user_similarity[a]
            
            pred_matrix[a][i] = mean_ratings_of_users[a] + sum
            
    return pred_matrix
        

def create_episode_matrix(user_item_matrix):
    episode_matrix = []
    r = user_item_matrix.shape[0]
    c = user_item_matrix.shape[1]
    for i in range(r):
        temp = []
        for j in range(c):
            if user_item_matrix[i][j] > 0:
                temp.append(user_item_matrix[i][j])
        episode_matrix.append(temp)
    return episode_matrix


def reward(j, i, episode_matrix, pred_matrix):
    ## episode_matrix contains rating of ith user, jth movie
    return episode_matrix[i][j+2] - pred_matrix[i][j]


def train_rlcf(episode_matrix, pred_matrix, alpha=0.1, discount=0.9):
    ## |S| = 10 [0.5, 5] 
    Q = np.zeros((10, 10))
    K = len(episode_matrix)
    
    for i in range(K): #users
        for j in range(len(episode_matrix[i])-2): #items
            r = reward(j, i, episode_matrix, pred_matrix)
            qi = int(episode_matrix[i][j])
            qj = int(episode_matrix[i][j+1])
            #Q[qi][qj] = Q[qi][qj] + alpha*(r + (discount * episode_matrix[i][j+1]) - Q[qi][qj])
            Q[qi][qj] = Q[qi][qj] + alpha*(r + (discount * max(episode_matrix[i])) - Q[qi][qj])
    return Q

def prediction(Q, episode_matrix, pred_matrix):
    P = np.zeros(pred_matrix.shape)
    
    K = len(episode_matrix)
    for i in range(K):
        for j in range(2, len(episode_matrix[i])):
            qi = int(episode_matrix[i][j-2])
            qj = int(episode_matrix[i][j-1])
            P[i][j] = pred_matrix[i][j] + Q[qi, qj]
    return P
            

In [31]:
def main():
    path = 'data/'
    data = merge_datasets(path)

    A = create_user_item_pair(data)
    I = create_matrix_I(A)
    uu_matrix = create_user_user_similarity_matrix(A)
    '''pred_matrix = create_prediction_matrix(A, uu_matrix)
    pickle_out = open('pred_matrix.pickle', 'wb')
    pickle.dump(pred_matrix, pickle_out)
    pickle_out.close()'''
    
    pickle_in = open('pred_matrix.pickle', 'rb')
    pred_matrix = pickle.load(pickle_in)
    
    episode_matrix = create_episode_matrix(A)
    
    Q = train_rlcf(episode_matrix, pred_matrix, alpha=0.000006, discount=0.65)
    
    P = prediction(Q, episode_matrix, pred_matrix) #predictor + Q

    #print(pred_matrix[0])
    
    

if __name__ == '__main__':
    main()

[3.3101449  2.223722   2.94294414 4.11305052 2.28944793 3.14325363
 2.41316839 3.73407788 2.70059981 3.7085211  2.45579913 3.55862921
 3.61230894 2.90197273 4.21759389 3.95713096 4.00531409 2.84257982
 3.01303003 3.95388625 2.955354   3.79300289 3.19939363 2.32731312
 4.65388757 2.81465758 2.56599901 4.00875295 2.85555544 4.18779276
 2.84026065 3.23109532 2.90484132 3.0004231  3.11117375 3.46922162
 4.12590976 3.47322797 3.21168549 1.48868958 2.41808239 2.63746416
 3.43402903 2.14112505 2.44107143 3.64190233 2.68668362 2.35488367
 2.91837376 2.56677554 3.94314621 2.24399643 1.77950502 3.48537128
 4.07610384 1.87376555 4.09433262 3.16460582 1.77880798 2.58558305
 2.40913644 3.89074807 3.95824126 2.75902772 2.28925364 2.47635295
 1.9492513  3.38143844 4.61090628 2.3728242  2.43452875 4.55702496
 4.21818046 1.88495318 4.22982447 2.37269665 3.48498573 2.40106703
 2.14626581 2.22323008 3.09274328 2.68673705 2.92444693 2.42641708
 3.0902561  3.13282801 2.59449267 3.90725314 2.5354253  2.9467